In [14]:
!pip install faiss-cpu sentence-transformers transformers

In [15]:
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch
from transformers import BertTokenizer, BertModel

In [16]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import json

# index the data with FAISS
# Step 2: Index the Data Using FAISS
def index_data(jsonl_file, index_file, metadata_file):
    # Load model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Load JSONL chunks (one JSON object per line)
    data_chunks = []
    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            data_chunks.append(json.loads(line))

    # Build texts to embed + metadata to save
    corpus_texts = []
    corpus_meta = []

    for chunk in data_chunks:
        # Build a natural-language string for the embedding
        parts = [ f'Pokemon: {chunk["pokemon"]}. Section: {chunk["section"]}.' ]

        # Some chunks have "description", others don't
        if chunk.get("description"):
            parts.append(chunk["description"])

        parts.append(chunk["text"])

        full_text = " ".join(parts)
        corpus_texts.append(full_text)

        # Store useful metadata for later use
        corpus_meta.append(
            {
                "id": chunk["id"],
                "pokemon": chunk["pokemon"],
                "section": chunk["section"],
                "text": chunk["text"],
                "description": chunk.get("description"),
                "metadata": chunk["metadata"],  # keep full structured metadata
            }
        )

    # Encode text chunks into embeddings
    embeddings = model.encode(corpus_texts, convert_to_tensor=False, show_progress_bar=True)

    # Create FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(np.array(embeddings, dtype="float32"))
    faiss.write_index(index, index_file)

    # Save metadata
    with open(metadata_file, "w", encoding="utf-8") as meta_f:
        json.dump(corpus_meta, meta_f, indent=4, ensure_ascii=False)

In [17]:
jsonl_file = "pokemon-test-1.jsonl"
index_file = "/content/pokemon_bert_faiss.index"
metadata_file = "/content/pokemon_bert_metadata.json"

index_data(jsonl_file, index_file, metadata_file)


Batches:   0%|          | 0/30 [00:00<?, ?it/s]